In [1]:
# Change all resource from input directory to working directory
# import shutil

# input_dir = "/kaggle/input/license-plate-recognition-resource"
# working_dir = "/kaggle/working/license-plate-recognition-resource"

# shutil.copytree(input_dir, working_dir, dirs_exist_ok=True)

In [2]:
import sys
import os
from pathlib import Path


# Add root directory of rhe project into sys.path (local)
root_dir = str(Path.cwd().parent.absolute())
if not root_dir in sys.path:
    sys.path.insert(0, root_dir)

# Add root directory of rhe project into sys.path (Kaggle)
# if working_dir not in sys.path:
#     sys.path.insert(0, working_dir)

In [3]:
# The below codes will work when running code again
# Copy all ouput files from the previous session to current session (include model checkpoints and downloaded datasets)

# input_dirs = [
#     d for d in os.listdir("/kaggle/input") if "license-plate-recognition-project" in d
# ]

# if input_dirs:
#     previous_session_output_dir = f"/kagge/input/{input_dirs[0]}/license-plate-recognition-resource"
#     shutil.copytree(
#         src=previous_session_output_dir,
#         dst=working_dir, 
#         dirs_exist_ok=True, 
#     )

#     print("Loaded outputs from previous version")
# else:
#     print("No previous outputs found. Starting the first session...")

## **0. Libraries**

In [4]:
from src.visualization import display_images_and_targets
from src.data_preprocessing import download_dataset, preprocess_data
from src.dataset import LicensePlateDataset
from src.model import FTFasterRCNN
from src.train import train_model
from src.evaluate import evaluate_model
from src.callbacks import EarlyStopping, ModelCheckpoints
from src.config import Config

from torch.utils.data import DataLoader
from torch.optim import SGD
from torch.optim.lr_scheduler import StepLR

import torchinfo
import albumentations as A
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

## **1. Dataset**

In [5]:
# Download dataset from Github
download_dataset(
    url=Config.DATA_URL, 
    dest=Config.RAW_DATA_DIR
)

In [6]:
# Data preprocessing
preprocess_data(
    data_path=Config.RAW_DATA_DIR, 
    dest=Config.PROCESSED_DATA_DIR, 
    image_size=Config.IMAGE_SIZE
)

Preprocessing data in train folder...: 100%|██████████| 20580/20580 [01:40<00:00, 204.93it/s]


## **2. Data preparing**

In [7]:
# Get the list of file paths in each folder
file_pathes = {}
for dir_name in os.listdir(Config.PROCESSED_DATA_DIR):
    dir_path = os.path.join(Config.PROCESSED_DATA_DIR, dir_name)
    file_pathes[dir_name] = []
    for filename in os.listdir(dir_path):
        file_pathes[dir_name].append(os.path.join(dir_path, filename))

In [8]:
# Transform images
train_transforms = A.Compose([
    A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5), 
    A.HueSaturationValue(hue_shift_limit=10, sat_shift_limit=20, val_shift_limit=10, p=0.3), 
    A.GaussNoise(std_range=(0.1, 0.2), per_channel=False, p=0.3), 
    A.MotionBlur(blur_limit=3, p=0.2), 
    A.RandomFog(fog_coef_range=(0.1, 0.3), alpha_coef=0.08, p=0.1), 
    A.Rotate(limit=5, p=0.3), 
    A.Resize(height=Config.IMAGE_SIZE[0], width=Config.IMAGE_SIZE[1]), 
    A.Normalize(mean=Config.MEAN_NORMALIZATION, std=Config.STD_NORMALIZATION), 
    A.ToTensorV2(), 
], bbox_params=A.BboxParams(
    format="pascal_voc", 
    label_fields=["labels"], 
    min_visibility=0.3
))

val_transforms = A.Compose([
    A.Resize(height=Config.IMAGE_SIZE[0], width=Config.IMAGE_SIZE[1]), 
    A.Normalize(mean=Config.MEAN_NORMALIZATION, std=Config.STD_NORMALIZATION), 
    A.ToTensorV2(), 
], bbox_params=A.BboxParams(
    format="pascal_voc", 
    label_fields=["labels"], 
))

In [9]:
# Declare dataset and dataloader
train_dataset = LicensePlateDataset(
    file_paths=file_pathes["train"], 
    transforms=train_transforms
)
valid_dataset = LicensePlateDataset(
    file_paths=file_pathes["valid"], 
    transforms=val_transforms
)
test_dataset = LicensePlateDataset(
    file_paths=file_pathes["test"]
)

In [10]:
def collate_fn(batch):
    """
    Custom collate function to handle batches with varying number of bounding boxes.
    """
    images, targets = zip(*batch)
    images = list(images)  # Keep images as a list
    targets = list(targets)  # Keep targets as a list of dictionaries
    return images, targets

In [11]:
# Split dataset to batches
train_dataloader = DataLoader(
    dataset=train_dataset,
    batch_size=Config.BATCH_SIZE,
    num_workers=Config.NUM_WORKERS,
    persistent_workers=True if Config.NUM_WORKERS > 0 else False, 
    pin_memory=True,
    shuffle=True,
    prefetch_factor=10 if Config.NUM_WORKERS > 0 else None,
    collate_fn=collate_fn
)

valid_dataloader = DataLoader(
    dataset=valid_dataset,
    batch_size=Config.BATCH_SIZE,
    num_workers=Config.NUM_WORKERS,
    persistent_workers=True if Config.NUM_WORKERS > 0 else False, 
    pin_memory=True,
    shuffle=True,
    prefetch_factor=10 if Config.NUM_WORKERS > 0 else None,
    collate_fn=collate_fn
)

test_dataloader = DataLoader(
    dataset=test_dataset,
    batch_size=Config.BATCH_SIZE,
    num_workers=Config.NUM_WORKERS,
    persistent_workers=True if Config.NUM_WORKERS > 0 else False, 
    pin_memory=True,
    shuffle=True,
    prefetch_factor=10 if Config.NUM_WORKERS > 0 else None,
    collate_fn=collate_fn
)

## **3. Faster RCNN model**

In [ ]:
faster_rcnn_model = FTFasterRCNN(num_classes=Config.NUM_CLASSES, freeze_backbone=True)
faster_rcnn_model.to(Config.DEVICE)
torchinfo.summary(faster_rcnn_model, (3, Config.IMAGE_SIZE[0], Config.IMAGE_SIZE[1]), batch_dim=0, depth=3)

In [ ]:
latest_checkpoint_filepath = f"{Config.CHECKPOINT_DIR}/latest_faster_rcnn_model.pt"
best_checkpoint_filepath= f"{Config.CHECKPOINT_DIR}/best_faster_rcnn_model.pt"

optimizer = SGD(
    filter(lambda p: p.requires_grad, faster_rcnn_model.parameters()),
    lr=Config.LEARNING_RATE, 
    momentum=Config.MOMENTUM,
    weight_decay=Config.WEIGHT_DECAY
)
lr_scheduler = StepLR(
    optimizer=optimizer, 
    step_size=Config.STEP_SIZE,
    gamma=Config.GAMMA
)
early_stopping = EarlyStopping(
    patience=Config.EARLY_STOPPING_PATIENCE, 
    min_delta=Config.MIN_DELTA
)
model_checkpoints = ModelCheckpoints(
    filepath=latest_checkpoint_filepath,
    save_best_only=True, 
    best_checkpoint_filepath=best_checkpoint_filepath
)

In [ ]:
faster_rcnn_history = train_model(
    model=faster_rcnn_model, 
    train_dataloader=train_dataloader,
    epoches=Config.EPOCHES,
    optimizer=optimizer,
    lr_scheduler=lr_scheduler,
    valid_dataloader=valid_dataloader,
    early_stopping=early_stopping,
    model_checkpoint=model_checkpoints,
    resume_from_checkpoint=latest_checkpoint_filepath if os.path.exists(latest_checkpoint_filepath) else None
)

In [ ]:
# Draw Loss and Accuracy (Cập nhật)
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Loss graph
axes[0].plot(faster_rcnn_history['train_loss'], label='Train Loss', marker='o')
if 'val_loss' in faster_rcnn_history and len(faster_rcnn_history['val_loss']) > 0:
    axes[0].plot(faster_rcnn_history['val_loss'], label='Validation Loss', marker='s')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].set_title('Training and Validation Loss')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Accuracy graph - CHỈ validation accuracy
axes[1].plot(faster_rcnn_history['val_acc'], label='Validation Accuracy', marker='s', color='orange')
if 'val_iou' in faster_rcnn_history and len(faster_rcnn_history['val_iou']) > 0:
    axes[1].plot(faster_rcnn_history['val_iou'], label='Validation IoU', marker='o', color='blue')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Score')
axes[1].set_title('Validation Metrics')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


## **4. Model evaluation**

In [ ]:
test_images, test_targets, test_predictions = evaluate_model(
    checkpoint_path=best_checkpoint_filepath, 
    test_dataloader=test_dataloader
)

In [ ]:
display_images_and_targets(test_images, test_predictions)